In [1]:
from klaviyo_api import KlaviyoAPI
from os import environ
from dotenv import load_dotenv
from openai import OpenAI
import json
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import requests
from typing import Any
from pydantic import BaseModel, EmailStr

# Load environment variables
load_dotenv()

True

In [2]:
### Clients
openai = OpenAI()
klaviyo = KlaviyoAPI(environ['KLAVIYO_API'], max_delay=60, max_retries=3, test_host=None)
spotify = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=environ['SPOTIPY_CLIENT_ID'], client_secret=environ['SPOTIPY_CLIENT_SECRET'], redirect_uri='http://127.0.0.1/callback', scope='playlist-modify-public'))

In [3]:
# Initialise Models
class User(BaseModel):
    id: str
    name: str
    email: EmailStr

class Flow(BaseModel):
    id: str
    name: str
    keywords: list[str]
    sample_playlist_url: str

MOCK_DB: dict[str, User] = {}
MOCK_FLOW_DB: dict[str, Flow] = {}

In [4]:
### Klaviyo API

# Populate Flow DB
def populate_mock_db():
    flows = klaviyo.Flows.get_flows()["data"]
    for flow in flows:
        MOCK_FLOW_DB[flow["id"]] = {
            "id": flow["id"], 
            "name": flow["attributes"]["name"], 
            "keywords": [],
            "sample_playlist_url":""}

populate_mock_db()

In [5]:
# Get Flow DB
# Get Mock Flow DB
def get_mock_flow_db():
    return MOCK_FLOW_DB

get_mock_flow_db()

{'STG99J': {'id': 'STG99J',
  'name': 'New Customer',
  'keywords': [],
  'sample_playlist_url': ''},
 'UxZKfm': {'id': 'UxZKfm',
  'name': 'Create Playlist',
  'keywords': [],
  'sample_playlist_url': ''},
 'VKztjN': {'id': 'VKztjN',
  'name': 'First-time purchase',
  'keywords': [],
  'sample_playlist_url': ''},
 'WQAhTH': {'id': 'WQAhTH',
  'name': 'SMS Welcome Series with Discount',
  'keywords': [],
  'sample_playlist_url': ''}}

In [6]:
### OpenAI API
keywords = "Funny, Heartwarming, Kind"

# Creating a playlist mood
completion = openai.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in writing concise yet emotional prose. Your role is to write a single sentence. You will have a set of keywords to create your concise, descriptive sentence. Based on the keywords, describe a mood. Do not use the word mood or genre. The description will be used for a music playlist to someone you care about. Do not use a description you have already used."},
    {"role": "user", "content": keywords}
  ]
)

mood = completion.choices[0].message.content
mood

'An infectious joy dances within your heart, painting laughter across your lips and unwrapping warmth in every act of kindness.'

In [7]:
# Creating a JSON playlist
response = openai.chat.completions.create(
  model="gpt-4-1106-preview",
  response_format={"type":"json_object"},
  messages=[
    {"role": "system", "content": "You are a helpful assistant designed to output perfectly formatted JSON. \
    You role is to return a list of song names and song artists. \
    You will have a description to create your list. \
    Based on the description, create a list of songs. \
    Do not use the word song or artist. \
    The list will be used for a music playlist to someone you care about. \
    Do not use a list you have already used."},
    {"role": "user", "content": mood}
  ]
)

gpt_playlist = json.loads(response.choices[0].message.content)
gpt_playlist 

{'playlist': [{'title': 'Happy', 'artist': 'Pharrell Williams'},
  {'title': "Can't Stop The Feeling!", 'artist': 'Justin Timberlake'},
  {'title': 'Uptown Funk', 'artist': 'Mark Ronson ft. Bruno Mars'},
  {'title': 'I Gotta Feeling', 'artist': 'The Black Eyed Peas'},
  {'title': 'Walking On Sunshine', 'artist': 'Katrina and The Waves'},
  {'title': 'Best Day of My Life', 'artist': 'American Authors'},
  {'title': 'Shake It Off', 'artist': 'Taylor Swift'},
  {'title': "Don't Worry Be Happy", 'artist': 'Bobby McFerrin'},
  {'title': 'On Top of the World', 'artist': 'Imagine Dragons'},
  {'title': 'Good as Hell', 'artist': 'Lizzo'}]}

In [8]:
# Creating an improved JSON playlist
response = openai.chat.completions.create(
        model="gpt-4-1106-preview",
        response_format={"type": "json_object"},
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant designed to output perfectly formatted JSON. \
                You role is to return a list of song names and song artists, as well as a playlist title. \
                You will have a description to create your list. \
                Based on the description, create a list of songs. \
                Do not use the word song or artist. \
                The list will be used for a music playlist to someone you care about. \
                Do not use a list you have already used. \
                There should be a key for the playlist_title and tracks. \
                Tracks should be a list of song titles and artist.",
            },
            {
                "role": "assistant",
                "content": "Are you sure that it is valid? Make sure there is a key for playlist_title and tracks. \
                tracks is a list of song and artist. \
                It should look like this: {\"playlist_title\": \"My Playlist\", \"tracks\": [{\"song\": \"Song Name\", \"artist\": \"Artist Name\"}, {\"song\": \"Song Name\", \"artist\": \"Artist Name\"}]}",
            },
            {
                "role": "user",
                "content": mood,
            },
        ],
    )

gpt_playlist = json.loads(response.choices[0].message.content)
gpt_playlist

{'playlist_title': 'Heartfelt Harmonies',
 'tracks': [{'title': 'Happy', 'artist': 'Pharrell Williams'},
  {'title': "Don't Worry, Be Happy", 'artist': 'Bobby McFerrin'},
  {'title': 'Walking on Sunshine', 'artist': 'Katrina and the Waves'},
  {'title': 'Lovely Day', 'artist': 'Bill Withers'},
  {'title': 'Good as Hell', 'artist': 'Lizzo'},
  {'title': 'I Got You (I Feel Good)', 'artist': 'James Brown'},
  {'title': "Can't Stop the Feeling!", 'artist': 'Justin Timberlake'},
  {'title': 'Uptown Funk', 'artist': 'Mark Ronson feat. Bruno Mars'},
  {'title': 'Shake It Off', 'artist': 'Taylor Swift'},
  {'title': 'Best Day of My Life', 'artist': 'American Authors'}]}

In [11]:
### Spotify API

# Create public playlist to populate
def create_playlist(user:str, title:str, description:str) -> dict:
    return spotify.user_playlist_create(user, title, description=description)

# Search and add tracks to playlist
def add_track_to_playlist(track:dict, playlist:dict):
    # Search for most similar track and artist
    artist = track['artist']
    title = track['title']
    spotify_track = spotify.search(q=f'artist:{artist} track:{title}', limit=1, type='track')
        
    # Add track to playlist
    track_id = spotify_track['tracks']['items'][0]['id']
    spotify.user_playlist_add_tracks(user, playlist['id'], [track_id]) 

# Create playlist
user = environ.get('SPOTIFY_USER')
title = gpt_playlist['playlist_title']
description = mood

spotify_playlist = create_playlist(user, title, description)

# Search for track with closest relevance
for song in gpt_playlist['tracks']:
    add_track_to_playlist(song, spotify_playlist)

# Finally, get playlist link
playlist_url = spotify_playlist['external_urls']['spotify']
playlist_description = spotify_playlist['description']
playlist_title = spotify_playlist['name']

print(playlist_title, playlist_description, playlist_url)

Heartfelt Harmonies An infectious joy dances within your heart, painting laughter across your lips and unwrapping warmth in every act of kindness. https://open.spotify.com/playlist/12Mlp5EmXExVBjcC7bgEbG


In [ ]:
### Klaviyo API

# Place an Order Event to trigger a First Purchased Flow
event_json = {
    "data": {
        "type": "event",
        "attributes": {
            "properties": {"Type": "Election", "amount": 500.00, "Category": "Annual"},
            "metric": {
                "data": {"type": "metric", "attributes": {"name": "Placed Order"}}
            },
            "profile": {
                "data": {
                    "type": "profile",
                    "attributes": {"email": "purple_cat@klaviyo-demo.com"},
                }
            },
        },
    }
}

url = "https://a.klaviyo.com/client/events/?company_id=UCR95t"
headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "revision": "2023-12-15",
}
response = requests.post(url, headers=headers, json=event_json)
response.status_code

In [ ]:
### Create an Event to trigger an Email Flow
event_json = {
    "data": {
        "type": "event",
        "attributes": {
            "properties": {"title": playlist_title, "url": playlist_url, "description": playlist_description},
            "metric": {
                "data": {"type": "metric", "attributes": {"name": "Playlist Created"}}
            },
            "profile": {
                "data": {
                "type":"profile",
                "attributes": {"email": "simon.karumbi@gmail.com"},
                }
            }
        },
    }
}

url = "https://a.klaviyo.com/client/events/?company_id=UCR95t"
headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "revision": "2023-12-15",
}
response = requests.post(url, headers=headers, json=event_json)
response.status_code

In [ ]:
# Klaviyo Webhook
url = f"{environ['NGROK_URL']}/webhooks/klaviyo"
event_json = {
    "flow_id": "VKztjN",
    "email": "simon.karumbi@gmail.com"
}
requests.post(url, json=event_json)

### Webserver
base_url = 'http://127.0.0.1:8000'
event_json = {
    "flow_id": "VKztjN",
    "email": "simon.karumbi@gmail.com"}


requests.post(f"{base_url}/webhooks/klaviyo", json=event_json)

In [ ]:
FLOW_PLAYLIST_DB = {}
FLOW_PLAYLIST_DB['STG99J'] = {'id': 'STG99J', 'name': 'New Customer', 'keywords': [], 'sample_playlist_url': '', 'active': False}

In [ ]:
# Insert into Flow Playlist Database
def insert_db(id: str, keywords: list[str], sample_playlist_url: str):
    if id in FLOW_PLAYLIST_DB:
        FLOW_PLAYLIST_DB[id].update(
            {
                "id": id,
                "keywords": keywords,
                "sample_playlist_url": sample_playlist_url,
                "active": True,
            }
        )
        print(f'updated {FLOW_PLAYLIST_DB[id]}!')
    return FLOW_PLAYLIST_DB